In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE168049"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE168049"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE168049.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE168049.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE168049.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prognosis associated mRNA and microRNA in peripheral blood mononuclear cells (PBMCs) from hepatitis B virus-related acute-on-chronic liver failure (HBV-ACLF)"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease: hepatitis B virus-related acute-on-chronic liver failure (HBV-ACLF)'], 1: ['tissue: whole blood'], 2: ['gender: male', 'gender: female'], 3: ['age: 35', 'age: 36', 'age: 57', 'age: 37', 'age: 58', 'age: 53', 'age: 30', 'age: 44', 'age: 69', 'age: 67', 'age: 34', 'age: 55', 'age: 62'], 4: ['survival state of 28-day: survivial', 'survival state of 28-day: dead']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Given the background information, this appears to be a dataset about HBV-ACLF with mRNA data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: looking at the sample characteristics, key 4 has survival state
trait_row = 4
# For age: key 3 has various ages
age_row = 3
# For gender: key 2 has gender information
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert survival state to binary: 0 for dead, 1 for survival."""
    if pd.isna(value) or value is None:
        return None
    value = value.lower()
    if "survival state of 28-day:" in value:
        value = value.replace("survival state of 28-day:", "").strip()
    if "survivial" in value or "survival" in value:
        return 1
    elif "dead" in value:
        return 0
    return None

def convert_age(value: str) -> float:
    """Convert age to continuous value."""
    if pd.isna(value) or value is None:
        return None
    if "age:" in value:
        try:
            return float(value.split("age:")[1].strip())
        except:
            return None
    return None

def convert_gender(value: str) -> int:
    """Convert gender to binary: 0 for female, 1 for male."""
    if pd.isna(value) or value is None:
        return None
    if "gender:" in value:
        value = value.replace("gender:", "").strip().lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# In this step, we're only analyzing the dataset, not processing it.
# Based on the error, accessing clinical_data.csv failed because it doesn't exist.
# We'll skip the actual extraction and save for now, as this step is just for analysis.

print(f"Dataset Analysis Summary:")
print(f"- Gene Expression Data Available: {is_gene_available}")
print(f"- Trait Data Available: {is_trait_available}")
print(f"- Age Data Available: {age_row is not None}")
print(f"- Gender Data Available: {gender_row is not None}")
print(f"- Trait is in row: {trait_row}")
print(f"- Age is in row: {age_row}")
print(f"- Gender is in row: {gender_row}")
print(f"Note: The actual clinical data processing will be done in a subsequent step.")


Dataset Analysis Summary:
- Gene Expression Data Available: True
- Trait Data Available: True
- Age Data Available: True
- Gender Data Available: True
- Trait is in row: 4
- Age is in row: 3
- Gender is in row: 2
Note: The actual clinical data processing will be done in a subsequent step.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 48908 rows
First 20 gene IDs:
Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315529', 'A_19_P00315541',
       'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315584',
       'A_19_P00315603', 'A_19_P00315625', 'A_19_P00315627', 'A_19_P00315631',
       'A_19_P00315641', 'A_19_P00315647', 'A_19_P00315649', 'A_19_P00315668'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyzing gene identifiers in the dataset

# The identifiers observed in the gene expression data (e.g., 'A_19_P00315452') 
# are Agilent microarray probe IDs, not standard human gene symbols.
# These are probe identifiers from an Agilent microarray platform.
# These identifiers need to be mapped to standard human gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE168049
Line 6: !Series_title = Prognosis associated mRNA and microRNA in peripheral blood mononuclear cells (PBMCs) from hepatitis B virus-related acute-on-chronic liver failure (HBV-ACLF)
Line 7: !Series_geo_accession = GSE168049
Line 8: !Series_status = Public on May 19 2021
Line 9: !Series_submission_date = Mar 02 2021
Line 10: !Series_last_update_date = May 19 2021
Line 11: !Series_pubmed_id = 33996909
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_type = Non-coding RNA profiling by array
Line 16: !Series_sample_id = GSM5124350
Lin


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_00110553

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene annotation store probe IDs and gene symbols
# Based on the preview, the 'ID' column matches the probe identifiers in the gene expression data
# and 'GENE_SYMBOL' contains the corresponding gene symbols
probe_column = 'ID'
gene_symbol_column = 'GENE_SYMBOL'

print(f"Using mapping from {probe_column} (probe IDs) to {gene_symbol_column} (gene symbols)")

# 2. Extract the gene mapping dataframe with the two columns
try:
    gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_column, gene_col=gene_symbol_column)
    print(f"Created gene mapping with {len(gene_mapping)} entries")
    print("First 5 mappings:")
    print(gene_mapping.head())
except Exception as e:
    print(f"Error creating gene mapping: {e}")
    
# 3. Apply the gene mapping to convert probe-level to gene-level expression
try:
    # Apply the gene mapping to convert probe-level measurements to gene expression
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Converted probe-level data to gene-level expression")
    print(f"Original probe count: {len(gene_mapping)}")
    print(f"Unique gene symbols after mapping: {len(gene_data)}")
    print("First 10 gene symbols after mapping:")
    print(gene_data.index[:10])
    
    # Save the gene expression data for later use
    gene_data.to_csv(out_gene_data_file)
    print(f"Saved gene expression data to {out_gene_data_file}")
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    is_gene_available = False


Using mapping from ID (probe IDs) to GENE_SYMBOL (gene symbols)
Created gene mapping with 48862 entries
First 5 mappings:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146
Converted probe-level data to gene-level expression
Original probe count: 48862
Unique gene symbols after mapping: 29222
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


Saved gene expression data to ../../output/preprocess/Hepatitis/gene_data/GSE168049.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (29222, 16)
Gene data shape after normalization: (20778, 16)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE168049.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE168049.csv
Linked data shape: (16, 20781)

Handling missing values...


After missing value handling, linked data shape: (16, 20781)

Evaluating feature bias...
For the feature 'Hepatitis', the least common label is '1.0' with 8 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Hepatitis' in this dataset is fine.

Quartiles for 'Age':
  25%: 36.0
  50% (Median): 54.0
  75%: 59.0
Min: 30.0
Max: 69.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 3 occurrences. This represents 18.75% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Trait bias evaluation result: False

Dataset usability: True


Linked data saved to ../../output/preprocess/Hepatitis/GSE168049.csv
